In [46]:
import pandas as pd
import sqlite3 
import geopy.distance
import folium
import geocoder
import html
import streamlit
import streamlit_folium
import dask.dataframe as dd
import altair as alt


In [3]:
g = geocoder.ip('me')
print(g.latlng)

2022-05-21 14:25:32.439 INFO    geocoder.base: Requested http://ipinfo.io/json


[22.6163, 120.3133]


In [4]:
#swifter.

In [5]:
conn = sqlite3.connect("./rapidtest.sqlite")

In [6]:
df = pd.read_sql("select * from rapidtest",conn, )

In [7]:
conn.close()

In [9]:
df.Time = pd.to_datetime(df.Time)

In [10]:
pharm_list = df.groupby("Code").last().reset_index()

In [12]:
df[df.Time == df.Time.max()]

,index,Code,Name,Addr,Longitude,Latitude,Tel,Brand,Stock,Time,Note
4725457,0,0291010010,連江縣立醫院,連江縣南竿鄉復興村２１７號,119.9492,26.16084,(083)623995,羅氏家用新冠病毒抗原自我檢測套組,199,2022-05-21 14:24:32,4／29起固定周一至周六上午8:30-9:30快篩試劑販售.
4725458,1,2331200010,新北市坪林區衛生所,新北市坪林區坪林街１０４號,121.7095,24.93592,(02)26656272,羅氏家用新冠病毒抗原自我檢測套組,144,2022-05-21 14:24:32,防疫口罩販售服務時間，星期一至五09:00-12:00 。（倘若遇政府規定補班日，仍會提供販...
4725459,2,2331220012,新北市石門區衛生所,新北市石門區中山路２８－１號,121.5660,25.29016,(02)26381007,羅氏家用新冠病毒抗原自我檢測套組,33,2022-05-21 14:24:32,端午節連假休診
4725460,3,2331240014,新北市平溪區衛生所,新北市平溪區公園街１７號１樓,121.7365,25.02469,(02)24951015,羅氏家用新冠病毒抗原自我檢測套組,105,2022-05-21 14:24:32,快篩試劑販售時間:週一至週五上午9:00至12:00 ，上午9:00開始發放號碼牌，限額78...
4725461,4,2331260016,新北市貢寮區衛生所,新北市貢寮區仁愛路１２８號,121.9095,25.02258,(02)24901431,羅氏家用新冠病毒抗原自我檢測套組,164,2022-05-21 14:24:32,快篩試劑販售時間為周一至周五早上9點開始販售，每日販售一箱，數量有限售完為止。
...,...,...,...,...,...,...,...,...,...,...,...
4729475,4018,5946012367,回春藥局,臺東縣台東市光明路１９６號,121.1507,22.75422,(089)322534,羅氏家用新冠病毒抗原自我檢測套組,7,2022-05-21 14:24:32,領取實名快篩時間 下午2點開始販售
4729476,4019,5946080552,伯安健康藥局,臺東縣長濱鄉長濱村１０鄰長濱路１０５號,121.4515,23.31496,(089)832519,羅氏家用新冠病毒抗原自我檢測套組,38,2022-05-21 14:24:32,無
4729477,4020,5946101118,伯安藥局,臺東縣池上鄉新生路１１６號,121.2187,23.12315,(089)863585,羅氏家用新冠病毒抗原自我檢測套組,41,2022-05-21 14:24:32,早0800-1200，午1400-1800，晚1800-2000
4729478,4021,5946101127,開心藥局,臺東縣池上鄉中山路１３６號,121.2158,23.12238,(089)861151,羅氏家用新冠病毒抗原自我檢測套組,14,2022-05-21 14:24:32,快篩實名制週一至週五中午12點開始販售，週六依人力彈性調整，週日休息


In [17]:
df=df.set_index("Time")

In [24]:
gdf = df.groupby("Code").resample('1D').Stock.agg(['max','min'])

In [37]:
gdf = gdf.fillna(0)

In [48]:
gdf['Sell'] = gdf['max']-gdf['min']

In [51]:
gdf = gdf.reset_index()

In [62]:
pharm_list.set_index('Code').Addr.str.slice(0,3)

Code
0291010010    連江縣
2331200010    新北市
2331220012    新北市
2331240014    新北市
2331260016    新北市
             ... 
5990010631    金門縣
5990010695    金門縣
5990020020    金門縣
5990030044    金門縣
5990030062    金門縣
Name: Addr, Length: 4964, dtype: object

In [52]:
gdf

,Code,Time,max,min,Sell
0,0291010010,2022-05-07,57.0,57.0,0.0
1,0291010010,2022-05-08,0.0,0.0,0.0
2,0291010010,2022-05-09,156.0,10.0,146.0
3,0291010010,2022-05-10,88.0,3.0,85.0
4,0291010010,2022-05-11,78.0,4.0,74.0
...,...,...,...,...,...
71462,5990030062,2022-05-14,75.0,4.0,71.0
71463,5990030062,2022-05-15,74.0,69.0,5.0
71464,5990030062,2022-05-16,0.0,0.0,0.0
71465,5990030062,2022-05-17,0.0,0.0,0.0


In [65]:
gdf = gdf.join(pharm_list.set_index('Code').Addr.str.slice(0,3), on='Code',)

### Sell

In [47]:
alt.data_transformers.enable('json')


DataTransformerRegistry.enable('json')

In [114]:


highlight = alt.selection(type='single', on='mouseover',
                          fields=['Addr'], nearest=True)
base = alt.Chart(gdf).mark_line().encode(
    x='Time:T',
    y='sum(Sell):Q',
    color='Addr:N',
    #row='Addr',
    tooltip=['Addr','sum(Sell):Q','Time'],
)


points = base.mark_circle().encode(
    opacity=alt.value(0)
).add_selection(
    highlight
).properties(
    width=400
)

lines = base.mark_line().encode(
    size=alt.condition(~highlight, alt.value(1), alt.value(2)),
    opacity=alt.condition(~highlight, alt.value(0.5), alt.value(1))
)

side = side = alt.Chart(gdf).mark_boxplot(extent='min-max').encode(
    x='Time:O',
    y='Sell:Q',
).add_selection(
    highlight
)

(points + lines)| side

alt.LayerChart(...)

In [106]:
single = alt.selection_single()
base = alt.Chart(gdf).mark_line().encode(
    x='Time:T',
    y='sum(Sell):Q',
    color=alt.condition(single, 'Addr:N', alt.value('lightgray')),
    #row='Addr',
    tooltip=['Addr','sum(Sell)','Time'],

)
side = alt.Chart(gdf).mark_boxplot(extent='min-max').encode(
    x='Time:O',
    y='Sell:Q',
    row='Addr',
)


base|side

alt.HConcatChart(...)

In [109]:
side = alt.Chart(gdf).mark_boxplot(extent='min-max').encode(
    x='monthdate(Time):O',
    y='Sell:Q',
    row='Addr',
)
side

alt.Chart(...)

In [94]:
single

Selection('selector025', SelectionDef({
  type: 'single'
}))

In [72]:
base = alt.Chart(gdf).mark_line().encode(
    x='Time:T',
    y='sum(Sell):Q',
    color='Addr:N',
    #row='Addr',
    #tooltip=['Code','max','min','Time'],
)
base

alt.Chart(...)

In [30]:
gdf.fillna(0).unstack().stack(level=0)

Time            2022-05-07  2022-05-08  2022-05-09  2022-05-10  2022-05-11  \
Code                                                                         
0291010010 max        57.0         0.0       156.0        88.0        78.0   
           min        57.0         0.0        10.0         3.0         4.0   
2331200010 max         NaN         NaN        78.0         0.0         0.0   
           min         NaN         NaN        31.0         0.0         0.0   
2331220012 max         NaN         NaN         NaN        76.0        78.0   
...                    ...         ...         ...         ...         ...   
5990020020 min         1.0         7.0         2.0         0.0         9.0   
5990030044 max        32.0         6.0         0.0         0.0        78.0   
           min        32.0         6.0         0.0         0.0        39.0   
5990030062 max        57.0        76.0         0.0         0.0         2.0   
           min        54.0         5.0         0.0         0.0         2.0   

Time            2022-05-12  2022-05-13  2022-05-14  2022-05-15  2022-05-16  \
Code                                                                         
0291010010 max        78.0        78.0        78.0         0.0       156.0   
           min         2.0         5.0         2.0         0.0        78.0   
2331200010 max         2.0        78.0        11.0        11.0        89.0   
           min         2.0        11.0        11.0        11.0        11.0   
2331220012 max        78.0        78.0         0.0         0.0        78.0   
...                    ...         ...         ...         ...         ...   
5990020020 min         8.0        51.0        26.0        37.0        15.0   
5990030044 max        73.0        78.0         0.0        77.0        77.0   
           min        43.0        39.0         0.0        43.0         5.0   
5990030062 max         0.0         0.0        75.0        74.0         0.0   
           min         0.0         0.0         4.0        69.0         0.0   

Time            2022-05-17  2022-05-18  2022-05-19  2022-05-20  2022-05-21  
Code                                                                        
0291010010 max       200.0       216.0       229.0       267.0       227.0  
           min       122.0       138.0       151.0       189.0       199.0  
2331200010 max       119.0       113.0       154.0       187.0       144.0  
           min        38.0        38.0        76.0       109.0       144.0  
2331220012 max        78.0        78.0        78.0        89.0        33.0  
...                    ...         ...         ...         ...         ...  
5990020020 min         0.0        24.0         0.0         2.0         2.0  
5990030044 max        71.0        69.0        72.0        78.0        73.0  
           min        11.0         1.0        20.0         1.0         7.0  
5990030062 max         0.0        78.0         NaN         NaN         NaN  
           min         0.0         8.0         NaN         NaN         NaN  

[9928 rows x 15 columns]

In [47]:
lat = 22.649242
lon = 120.310989
center = (lat, lon)

In [48]:
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1456786 entries, 0 to 1456785
Data columns (total 11 columns):
 #   Column     Non-Null Count    Dtype         
---  ------     --------------    -----         
 0   index      1456786 non-null  int64         
 1   Code       1456786 non-null  object        
 2   Name       1456786 non-null  object        
 3   Addr       1456786 non-null  object        
 4   Longitude  1456786 non-null  float64       
 5   Latitude   1456786 non-null  float64       
 6   Tel        1456786 non-null  object        
 7   Brand      1456786 non-null  object        
 8   Stock      1456786 non-null  int64         
 9   Time       1456786 non-null  datetime64[ns]
 10  Note       1456786 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(2), object(6)
memory usage: 122.3+ MB


In [49]:
%%time
pharm_list['Distance'] = pharm_list.apply(lambda x: geopy.distance.distance((x.Latitude,x.Longitude), g.latlng).km, axis=1)

CPU times: user 1.13 s, sys: 0 ns, total: 1.13 s
Wall time: 1.13 s


In [50]:
pharm_list

,Code,index,Name,Addr,Longitude,Latitude,Tel,Brand,Stock,Time,Note,Distance
0,0291010010,0,連江縣立醫院,連江縣南竿鄉復興村２１７號,119.9492,26.16084,(083)623995,羅氏家用新冠病毒抗原自我檢測套組,2,2022-05-14 09:34:32,4／29起固定周一至周六上午8:30-9:30快篩試劑販售.,394.340485
1,2331200010,0,新北市坪林區衛生所,新北市坪林區坪林街１０４號,121.7095,24.93592,(02)26656272,羅氏家用新冠病毒抗原自我檢測套組,11,2022-05-15 06:59:32,防疫口罩販售服務時間，星期一至五09:00-12:00 。（倘若遇政府規定補班日，仍會提供販...,293.684134
2,2331220012,1,新北市石門區衛生所,新北市石門區中山路２８－１號,121.5660,25.29016,(02)26381007,羅氏家用新冠病毒抗原自我檢測套組,1,2022-05-13 09:39:32,無,322.427138
3,2331240014,1,新北市平溪區衛生所,新北市平溪區公園街１７號１樓,121.7365,25.02469,(02)24951015,羅氏家用新冠病毒抗原自我檢測套組,10,2022-05-13 10:09:32,快篩試劑販售時間:週一至週五上午9:00至12:00 ，上午9:00開始發放號碼牌，限額78...,303.604381
4,2331260016,1,新北市貢寮區衛生所,新北市貢寮區朝陽街７０巷１０號、新北市貢寮區仁愛路１２８號,121.9095,25.02258,(02)24901431,羅氏家用新冠病毒抗原自我檢測套組,1,2022-05-13 10:14:32,快篩試劑販售時間為周一至周五早上9點開始販售，每日販售一箱，數量有限售完為止。,312.206967
...,...,...,...,...,...,...,...,...,...,...,...,...
4932,5990010631,1322,大森藥局,金門縣金城鎮民生路２８、３０號１、２樓,118.3195,24.43470,(82)325100,羅氏家用新冠病毒抗原自我檢測套組,78,2022-05-15 07:59:32,可直接憑健保卡買口罩,286.371925
4933,5990010695,2238,百泰藥局,金門縣金城鎮西海路１段１號,118.2864,24.41566,(082)312832,羅氏家用新冠病毒抗原自我檢測套組,78,2022-05-14 16:04:32,快篩實名制，每天16:00開始販售，售完為止。,287.328817
4934,5990020020,1362,大金藥局,金門縣金沙鎮汶沙里五福街１號,118.4130,24.48834,(082)355382,羅氏家用新冠病毒抗原自我檢測套組,37,2022-05-15 08:09:32,（以上口罩餘額很不準 詳情請洽藥局）每日早上8:00發號碼牌並於下午13:30後再來領口罩,283.945809
4935,5990030044,1368,仁愛復興藥局,金門縣金湖鎮新市里復興路４０號,118.4163,24.44123,(082)332368,羅氏家用新冠病毒抗原自我檢測套組,43,2022-05-15 08:14:32,實名制快篩於早上8點左右開始販售,279.949454


In [51]:
sdf = df[df.Code.isin(pharm_list[pharm_list.Distance<5].Code)].groupby(['Code','Name','Addr','Tel',"Latitude","Longitude",'Note','Time']).Stock.first().unstack().fillna(0).astype(int).iloc[:,-1:].sort_values(by=df.Time.max(), ascending=False)

In [52]:
df[df.Code.isin(pharm_list[pharm_list.Distance<5].Code)].groupby(['Code','Name','Addr','Tel',"Latitude","Longitude",'Note','Time']).Stock.first().unstack().T

Code                                                    5902010025  \
Name                                                          南京藥局   
Addr                                                高雄市鹽埕區建國四路３８７號   
Tel                                                    (07)5514652   
Latitude                                                  22.63279   
Longitude                                                 120.2840   
Note                早上9:00實名制口罩販賣；實名制快篩星期1-6為下午1:00開始販賣，星期日休息，售完為止   
Time                                                                 
2022-05-07 01:04:32                                            NaN   
2022-05-07 01:34:32                                            NaN   
2022-05-07 02:04:32                                            NaN   
2022-05-07 02:34:32                                            NaN   
2022-05-07 03:04:32                                            NaN   
...                                                            ...   
2022-05-15 12:34:32                                            NaN   
2022-05-15 12:39:32                                            NaN   
2022-05-15 12:44:32                                            NaN   
2022-05-15 12:49:32                                            NaN   
2022-05-15 12:54:34                                            NaN   

Code                                                        5902010132  \
Name                                                              宏恩藥局   
Addr                                                  高雄市鹽埕區教仁里瀨南街１３３號   
Tel                                                        (07)5211318   
Latitude                                                      22.62472   
Longitude                                                     120.2823   
Note                5／10 10:30貨到開始販售 星期一至星期六10:30貨到開始販售，星期日14:00貨到開始販售   
Time                                                                     
2022-05-07 01:04:32                                                NaN   
2022-05-07 01:34:32                                                NaN   
2022-05-07 02:04:32                                                NaN   
2022-05-07 02:34:32                                                NaN   
2022-05-07 03:04:32                                                NaN   
...                                                                ...   
2022-05-15 12:34:32                                                NaN   
2022-05-15 12:39:32                                                NaN   
2022-05-15 12:44:32                                                NaN   
2022-05-15 12:49:32                                                NaN   
2022-05-15 12:54:34                                                NaN   

Code                                                             \
Name                                                              
Addr                                                              
Tel                                                               
Latitude                                                          
Longitude                                                         
Note                5／6 10:00開始販售 星期一至星期六10:00開始販售，星期日14:00開始販售   
Time                                                              
2022-05-07 01:04:32                                        36.0   
2022-05-07 01:34:32                                        36.0   
2022-05-07 02:04:32                                        36.0   
2022-05-07 02:34:32                                        36.0   
2022-05-07 03:04:32                                        36.0   
...                                                         ...   
2022-05-15 12:34:32                                         NaN   
2022-05-15 12:39:32                                         NaN   
2022-05-15 12:44:32                                         NaN   
2022-05-15 12:49:32                                         NaN   
2022-05-15 12:54:34                                     

In [53]:
ts = df[df.Code.isin(pharm_list[pharm_list.Distance<5].Code)].groupby(['Code','Name','Addr','Tel',"Latitude","Longitude",'Note','Time']).Stock.first().unstack().fillna(0).astype(int).iloc[:,-30:].stack()

In [54]:
ts = ts.reset_index()

In [55]:
ts.columns = ['Code', 'Name', 'Addr', 'Tel', 'Latitude', 'Longitude', 'Note', 'Time',"Stock"]

In [56]:
import altair as alt

In [57]:
alt.Chart(ts[ts.Code=='5907300784']).mark_line().encode(x='Time',y='Stock')

alt.Chart(...)

In [175]:
ts[ts.Code=='5907300784']

,Code,Name,Addr,Tel,Latitude,Longitude,Note,Time,Stock
5610,5907300784,新高橋藥局博正店,高雄市左營區忠言路１８０號,(07)5509810,22.65841,120.3038,下午2:30開始販售口罩,2022-05-14 19:54:32,11
5611,5907300784,新高橋藥局博正店,高雄市左營區忠言路１８０號,(07)5509810,22.65841,120.3038,下午2:30開始販售口罩,2022-05-14 19:59:32,11
5612,5907300784,新高橋藥局博正店,高雄市左營區忠言路１８０號,(07)5509810,22.65841,120.3038,下午2:30開始販售口罩,2022-05-14 20:04:33,11
5613,5907300784,新高橋藥局博正店,高雄市左營區忠言路１８０號,(07)5509810,22.65841,120.3038,下午2:30開始販售口罩,2022-05-14 20:09:32,10
5614,5907300784,新高橋藥局博正店,高雄市左營區忠言路１８０號,(07)5509810,22.65841,120.3038,下午2:30開始販售口罩,2022-05-14 20:14:35,10
5615,5907300784,新高橋藥局博正店,高雄市左營區忠言路１８０號,(07)5509810,22.65841,120.3038,下午2:30開始販售口罩,2022-05-14 20:19:32,10
5616,5907300784,新高橋藥局博正店,高雄市左營區忠言路１８０號,(07)5509810,22.65841,120.3038,下午2:30開始販售口罩,2022-05-14 20:24:32,10
5617,5907300784,新高橋藥局博正店,高雄市左營區忠言路１８０號,(07)5509810,22.65841,120.3038,下午2:30開始販售口罩,2022-05-14 20:29:33,10
5618,5907300784,新高橋藥局博正店,高雄市左營區忠言路１８０號,(07)5509810,22.65841,120.3038,下午2:30開始販售口罩,2022-05-14 20:34:32,10
5619,5907300784,新高橋藥局博正店,高雄市左營區忠言路１８０號,(07)5509810,22.65841,120.3038,下午2:30開始販售口罩,2022-05-14 20:39:32,10


In [58]:
sdf= sdf.reset_index()

In [59]:
#sdf = df[df.Distance<2].groupby("Code").last()

In [60]:
sdf.columns[-1]

Timestamp('2022-05-15 12:54:34')

In [61]:
for i in sdf.Note:
    print(len(i),html.escape(i))


23 快篩實名制，每天16:00開始販售，售完為止。
23 快篩實名制，每天16:00開始販售，售完為止。
23 快篩實名制，每天16:00開始販售，售完為止。
23 快篩實名制，每天16:00開始販售，售完為止。
28 快篩實名制，每天下午1:00開始販售，現場排隊，售完為止
36 每天15:30發號碼牌，於18:00前兌換完畢，逾期棄權，剩餘則繼續發放
52 位於美麗島捷運站內，往西子灣方向地下二樓。快篩試劑正常到貨時，平日10:00販售 、假日13:00販售。
23 快篩實名制，每天16:00開始販售，售完為止。
23 快篩實名制，每天16:00開始販售，售完為止。
32 每天下午2點開始販售實名制快篩，售完為止，排隊一人限購一份~謝謝
23 快篩實名制，每天16:00開始販售，售完為止。
23 快篩實名制，每天16:00開始販售，售完為止。
23 快篩實名制，每天16:00開始販售，售完為止。
23 快篩實名制，每天16:00開始販售，售完為止。
23 快篩實名制，每天16:00開始販售，售完為止。
23 快篩實名制，每天16:00開始販售，售完為止。
28 2／11早上9:00至下午15:00，2／12至16日休
16 快篩每日下午1點開賣，售完為止.
21 中午12:00販售現場排隊（仕康健保藥局）
53 營業日0900可先領取號碼牌，上午9:00-12:00及下午3:00~10:.00為販賣快篩時間，售完為止
18 快篩試劑代售時間自早上9:00點服務
32 星期一~六早上9:00發號碼牌，12:00過後拿號碼牌買快篩試劑
29 快篩實名制上午9點30分開始販售，販售前15分鐘發號碼牌.
1 無
32 快篩試劑中午12點準時開賣賣完為止（販賣方式以藥局門口公告為主）
34 自2／11日~2／29日，營業日08:00~10:00為販賣口罩時間
19 9:00發放實名制快篩號碼牌.售完為止
20 快篩發放時間早上08:30開始 售完為止
11 下午3點販售，現場排隊
46 快篩於早上8:30領取號碼牌後販賣，請從後花園排隊。星期日下午5:00開始販賣，不領號碼牌。
12 下午2:30開始販售口罩
21 上午12點開賣.現場排隊販售.不採取號碼牌
15 快篩試紙領用時間為每日下午2點
27 賣快篩時間：周一至週日下午1:00開始販售，售完為止。
18 週一 

In [62]:
ph = df[df.Code.isin(pharm_list[pharm_list.Distance<5].Code)]

In [65]:
ph.Code.nunique()

236

In [74]:
data = ph.groupby(["Code","Name",'Time']).Stock.first().unstack().fillna(0)

In [75]:
data = data.stack().reset_index()

In [76]:
data.columns = ['Code', 'Name', 'Time', 'Stock']

In [77]:
sdata = data[data.Code=='5902010025']
sdata['Date']=sdata.Time.apply(lambda t:str(t.date()))
sdata['Hour']=sdata.Time.apply(lambda t:f"{t:%H:%M}")

/tmp/ipykernel_145017/482416467.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sdata['Date']=sdata.Time.apply(lambda t:str(t.date()))
/tmp/ipykernel_145017/482416467.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sdata['Hour']=sdata.Time.apply(lambda t:f"{t:%H:%M}")


In [78]:
sdata

,Code,Name,Time,Stock,Date,Hour
0,5902010025,南京藥局,2022-05-07 01:04:32,0.0,2022-05-07,01:04
1,5902010025,南京藥局,2022-05-07 01:34:32,0.0,2022-05-07,01:34
2,5902010025,南京藥局,2022-05-07 02:04:32,0.0,2022-05-07,02:04
3,5902010025,南京藥局,2022-05-07 02:34:32,0.0,2022-05-07,02:34
4,5902010025,南京藥局,2022-05-07 03:04:32,0.0,2022-05-07,03:04
...,...,...,...,...,...,...
1319,5902010025,南京藥局,2022-05-15 12:34:32,0.0,2022-05-15,12:34
1320,5902010025,南京藥局,2022-05-15 12:39:32,0.0,2022-05-15,12:39
1321,5902010025,南京藥局,2022-05-15 12:44:32,0.0,2022-05-15,12:44
1322,5902010025,南京藥局,2022-05-15 12:49:32,0.0,2022-05-15,12:49


In [72]:
sdata = sdata.groupby(['Date','Hour']).Stock.mean().reset_index()

KeyError: 'Time'

In [ ]:
sdata

,Date,Hour,Stock
0,2022-05-07,01:04,0.0
1,2022-05-07,01:34,0.0
2,2022-05-07,02:04,0.0
3,2022-05-07,02:34,0.0
4,2022-05-07,03:04,0.0
...,...,...,...
1319,2022-05-15,12:34,0.0
1320,2022-05-15,12:39,0.0
1321,2022-05-15,12:44,0.0
1322,2022-05-15,12:49,0.0


In [73]:
sdata.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1324 entries, 0 to 1323
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Date    1324 non-null   object 
 1   Hour    1324 non-null   object 
 2   Stock   1324 non-null   float64
dtypes: float64(1), object(2)
memory usage: 31.2+ KB


In [197]:
sdata.Date = sdata.Date.astype(str)

In [101]:
sdata

,Code,Name,Time,Stock,Date,Hour
0,5902010025,南京藥局,2022-05-07 01:04:32,0.0,2022-05-07,01:04
1,5902010025,南京藥局,2022-05-07 01:34:32,0.0,2022-05-07,01:34
2,5902010025,南京藥局,2022-05-07 02:04:32,0.0,2022-05-07,02:04
3,5902010025,南京藥局,2022-05-07 02:34:32,0.0,2022-05-07,02:34
4,5902010025,南京藥局,2022-05-07 03:04:32,0.0,2022-05-07,03:04
...,...,...,...,...,...,...
1319,5902010025,南京藥局,2022-05-15 12:34:32,0.0,2022-05-15,12:34
1320,5902010025,南京藥局,2022-05-15 12:39:32,0.0,2022-05-15,12:39
1321,5902010025,南京藥局,2022-05-15 12:44:32,0.0,2022-05-15,12:44
1322,5902010025,南京藥局,2022-05-15 12:49:32,0.0,2022-05-15,12:49


In [113]:


highlight = alt.selection(type='single', on='mouseover',
                          fields=['Date'], nearest=True)
base = alt.Chart(sdata).mark_line().encode(
    x='hoursminutes(Time):O',
    y='sum(Stock):Q',
    color='monthdate(Time):O',
    tooltip=['Stock','Date']
)



points = base.mark_circle().encode(
    opacity=alt.value(0)
).add_selection(
    highlight
).properties(
    width=1024
)

lines = base.mark_line().encode(
    size=alt.condition(~highlight, alt.value(1), alt.value(2)),
    opacity=alt.condition(~highlight, alt.value(0.5), alt.value(1))
)

(points + lines)

alt.LayerChart(...)

In [111]:
alt.Chart(df.iloc[:5100000]).mark_rect().encode(
    alt.X('hoursminutes(Time):O', title='hour of day'),
    alt.Y('monthdate(Time):O', title='date'),
    alt.Color('sum(Stock):Q', title='stock')
)

alt.Chart(...)

In [35]:
ss = pd.read_sql(f"select * from rapidtest where Code==5902050592",conn, )

In [97]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1456786 entries, 0 to 1456785
Data columns (total 11 columns):
 #   Column     Non-Null Count    Dtype         
---  ------     --------------    -----         
 0   index      1456786 non-null  int64         
 1   Code       1456786 non-null  object        
 2   Name       1456786 non-null  object        
 3   Addr       1456786 non-null  object        
 4   Longitude  1456786 non-null  float64       
 5   Latitude   1456786 non-null  float64       
 6   Tel        1456786 non-null  object        
 7   Brand      1456786 non-null  object        
 8   Stock      1456786 non-null  int64         
 9   Time       1456786 non-null  datetime64[ns]
 10  Note       1456786 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(2), object(6)
memory usage: 122.3+ MB


In [102]:
alt.data_transformers.enable('json')
#alt.data_transformers.enable('data_server')

DataTransformerRegistry.enable('json')

In [36]:
ss.Time = pd.to_datetime(ss.Time)

In [37]:
ss = ss.set_index("Time")

In [38]:
ss.resample("10Min").mean().fillna(0)

,index,Longitude,Latitude,Stock
Time,,,,
2022-05-07 09:30:00,210.0,120.3105,22.64469,58.0
2022-05-07 09:40:00,0.0,0.0000,0.00000,0.0
2022-05-07 09:50:00,0.0,0.0000,0.00000,0.0
2022-05-07 10:00:00,184.0,120.3105,22.64469,46.0
2022-05-07 10:10:00,0.0,0.0000,0.00000,0.0
...,...,...,...,...
2022-05-14 21:10:00,270.0,120.3105,22.64469,1.0
2022-05-14 21:20:00,266.5,120.3105,22.64469,1.0
2022-05-14 21:30:00,264.5,120.3105,22.64469,1.0


In [48]:
alt.Chart(ss.resample("5Min").first().interpolate(axis=1).reset_index()).mark_line().encode(x='Time',y='Stock')

TypeError: Cannot interpolate with all object-dtype columns in the DataFrame. Try setting at least one column to a numeric dtype.

In [61]:
si = ss.resample("5Min").first().interpolate(method='linear', limit=2, inplace=False, limit_direction='forward', limit_area='inside', downcast=None,).fillna(0)

In [62]:
si

,index,Code,Name,Addr,Longitude,Latitude,Tel,Brand,Stock,Note
Time,,,,,,,,,,
2022-05-07 09:35:00,210.000000,5902050592,建昌藥局,高雄市三民區十全一路１０５號１樓,120.3105,22.64469,(07)3232356,羅氏家用新冠病毒抗原自我檢測套組,58.0,賣口罩時間：周一至週五下午1:00起，周六至週日上午10:00起，售完為止。
2022-05-07 09:40:00,205.666667,0,0,0,120.3105,22.64469,0,0,56.0,0
2022-05-07 09:45:00,201.333333,0,0,0,120.3105,22.64469,0,0,54.0,0
2022-05-07 09:50:00,0.000000,0,0,0,0.0000,0.00000,0,0,0.0,0
2022-05-07 09:55:00,0.000000,0,0,0,0.0000,0.00000,0,0,0.0,0
...,...,...,...,...,...,...,...,...,...,...
2022-05-14 21:30:00,266.000000,5902050592,建昌藥局,高雄市三民區十全一路１０５號１樓,120.3105,22.64469,(07)3232356,羅氏家用新冠病毒抗原自我檢測套組,1.0,賣快篩時間：周一至週日下午1:00開始販售，售完為止。
2022-05-14 21:35:00,263.000000,5902050592,建昌藥局,高雄市三民區十全一路１０５號１樓,120.3105,22.64469,(07)3232356,羅氏家用新冠病毒抗原自我檢測套組,1.0,賣快篩時間：周一至週日下午1:00開始販售，售完為止。
2022-05-14 21:40:00,260.000000,5902050592,建昌藥局,高雄市三民區十全一路１０５號１樓,120.3105,22.64469,(07)3232356,羅氏家用新冠病毒抗原自我檢測套組,1.0,賣快篩時間：周一至週日下午1:00開始販售，售完為止。


In [63]:
alt.Chart(si.reset_index()).mark_line().encode(
    x='Time',
    y='Stock'
    
    )

alt.Chart(...)